In [1]:
from array import array
import subprocess
import math
import ROOT
import os
import numpy as np

Welcome to JupyROOT 6.29/01


In [2]:
buildcuda = '/home/mdessole/Projects/ROOT/genvectorx/build_cuda/'

In [3]:
buildoneapi = '/home/mdessole/Projects/ROOT/genvectorx/build_oneapi/'

In [4]:
buildopensycl = '/home/mdessole/Projects/ROOT/genvectorx/build_opensycl/'

In [5]:
!ls /home/mdessole/Projects/ROOT/genvectorx/build_opensycl/testx

CMakeFiles	     InvariantMasses	  SInvariantMasses
cmake_install.cmake  InvariantMassesSYCL  SInvariantMassesSYCL


In [6]:
!ls /home/mdessole/Projects/ROOT/genvectorx/build_oneapi/testx

CMakeFiles	     InvariantMasses	  SInvariantMasses
cmake_install.cmake  InvariantMassesSYCL  SInvariantMassesSYCL


In [7]:
!echo $LD_LIBRARY_PATH

/opt/intel/oneapi/tbb/2021.10.0/env/../lib/intel64/gcc4.8:/opt/intel/oneapi/debugger/2023.2.0/gdb/intel64/lib:/opt/intel/oneapi/debugger/2023.2.0/libipt/intel64/lib:/opt/intel/oneapi/debugger/2023.2.0/dep/lib:/opt/intel/oneapi/compiler/2023.2.1/linux/lib:/opt/intel/oneapi/compiler/2023.2.1/linux/lib/x64:/opt/intel/oneapi/compiler/2023.2.1/linux/lib/oclfpga/host/linux64/lib:/opt/intel/oneapi/compiler/2023.2.1/linux/compiler/lib/intel64_lin:/home/mdessole/Projects/sycl-blas/sycl-blas_build/benchmark/googlebench-build/src:/opt/OpenBLAS/lib:/home/mdessole/Projects/sycl-blas/sycl-blas_build/lib:/home/mdessole/Projects/sycl-blas/sycl-blas/lib:/home/mdessole/Projects/OpenSYCL/OpenSYCL_install/lib:/usr/local/cuda-11.8/lib64:/home/mdessole/Projects/ROOT/root/lib


In [8]:
sizes = array('d',[1000, 10000, 100000,1000000,5000000,10000000, 50000000,100000000]) #10,100,1000,10000,100000,
n = len(sizes)

In [9]:
Ssizes = array('d',[1000, 10000, 100000,1000000,5000000,10000000, 50000000,100000000,200000000]) #10,100,1000,10000,100000,
n = len(sizes)

In [10]:
def result2list(exe, N, nruns):
    result = subprocess.run([exe, str(int(N)), str(int(nruns)) ], stdout=subprocess.PIPE).stdout.decode('utf-8')
    print(result)
    if len(result)<=0:
        return []
    result = result.replace("\r","").replace("\n","")
    time = np.zeros(nruns)
    for r in range(nruns):
        i = result.find("time")
        j = result.find("(s)")
        if (i<0) or (j<0):
            break
        time[r] = float(result[i+5:j])
        result = result[j+3:]
    #endif
    return time

In [11]:
def run_benchmark(builddir, sizes, nruns = 1, testname = "InvariantMasses", btype = 'cpu'):
    timeopt = array( 'd' )
    timestd = array( 'd' )
    
    if btype == 'cpu':
        exe = os.path.join(builddir,'testx',testname)
    elif btype == 'cuda':
        exe = os.path.join(builddir,'testx',testname+'CUDA')
    elif btype == 'sycl':
        exe = os.path.join(builddir,'testx',testname+'SYCL')
    #endif
    
    if (not os.path.exists(exe)):
        print("Executable", exe, "not found!")
        return timeopt, timestd
    
    for i, N in enumerate(sizes):
        time = result2list(exe,N,nruns)
        if (len(time)<=0):
            return timeopt, timestd
        print(time, '\n')
        timeopt.append(np.amin(time))
        timestd.append(np.std(time)) 
    #endfor
    return timeopt, timestd

In [12]:
timecpu, stdcpu = run_benchmark(buildoneapi, sizes, nruns = 3, btype = 'cpu')

RVec cpu time 6.1e-05 (s)
RVec cpu time 6e-05 (s)
RVec cpu time 6.1e-05 (s)

[6.1e-05 6.0e-05 6.1e-05] 

RVec cpu time 0.000656 (s)
RVec cpu time 0.000595 (s)
RVec cpu time 0.000597 (s)

[0.000656 0.000595 0.000597] 

RVec cpu time 0.006061 (s)
RVec cpu time 0.006131 (s)
RVec cpu time 0.006067 (s)

[0.006061 0.006131 0.006067] 

RVec cpu time 0.064058 (s)
RVec cpu time 0.06123 (s)
RVec cpu time 0.060912 (s)

[0.064058 0.06123  0.060912] 

RVec cpu time 0.320743 (s)
RVec cpu time 0.322692 (s)
RVec cpu time 0.320889 (s)

[0.320743 0.322692 0.320889] 

RVec cpu time 0.64048 (s)
RVec cpu time 0.642431 (s)
RVec cpu time 0.643243 (s)

[0.64048  0.642431 0.643243] 

RVec cpu time 3.20529 (s)
RVec cpu time 3.21185 (s)
RVec cpu time 3.21986 (s)

[3.20529 3.21185 3.21986] 

RVec cpu time 6.50963 (s)
RVec cpu time 6.4768 (s)
RVec cpu time 6.48321 (s)

[6.50963 6.4768  6.48321] 



In [13]:
timecpu

array('d', [6e-05, 0.000595, 0.006061, 0.060912, 0.320743, 0.64048, 3.20529, 6.4768])

In [14]:
Stimecpu, Sstdcpu = run_benchmark(buildoneapi, Ssizes, testname = "SInvariantMasses", nruns = 3, btype = 'cpu')

SINGLE_PRECISION defined 
RVec cpu time 4.6e-05 (s)
RVec cpu time 4.6e-05 (s)
RVec cpu time 4.5e-05 (s)

[4.6e-05 4.6e-05 4.5e-05] 

SINGLE_PRECISION defined 
RVec cpu time 0.000448 (s)
RVec cpu time 0.000459 (s)
RVec cpu time 0.000463 (s)

[0.000448 0.000459 0.000463] 

SINGLE_PRECISION defined 
RVec cpu time 0.004544 (s)
RVec cpu time 0.004514 (s)
RVec cpu time 0.00454 (s)

[0.004544 0.004514 0.00454 ] 

SINGLE_PRECISION defined 
RVec cpu time 0.046482 (s)
RVec cpu time 0.045142 (s)
RVec cpu time 0.044971 (s)

[0.046482 0.045142 0.044971] 

SINGLE_PRECISION defined 
RVec cpu time 0.234015 (s)
RVec cpu time 0.232984 (s)
RVec cpu time 0.23485 (s)

[0.234015 0.232984 0.23485 ] 

SINGLE_PRECISION defined 
RVec cpu time 0.477016 (s)
RVec cpu time 0.472026 (s)
RVec cpu time 0.47033 (s)

[0.477016 0.472026 0.47033 ] 

SINGLE_PRECISION defined 
RVec cpu time 2.3574 (s)
RVec cpu time 2.3605 (s)
RVec cpu time 2.36316 (s)

[2.3574  2.3605  2.36316] 

SINGLE_PRECISION defined 
RVec cpu time 4.69

In [15]:
Stimecpu

array('d', [4.5e-05, 0.000448, 0.004514, 0.044971, 0.232984, 0.47033, 2.3574, 4.68996, 9.37755])

In [16]:
timeoneapi, stdoneapi = run_benchmark(buildoneapi, sizes, nruns = 3, btype = 'sycl')

ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000389 (s)
sycl time 5.7e-05 (s)
sycl time 0.000162 (s)

[3.89e-04 5.70e-05 1.62e-04] 

ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000446 (s)
sycl time 0.000111 (s)
sycl time 0.000193 (s)

[0.000446 0.000111 0.000193] 

ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.001427 (s)
sycl time 0.001098 (s)
sycl time 0.001005 (s)

[0.001427 0.001098 0.001005] 

ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.009814 (s)
sycl time 0.009204 (s)
sycl time 0.009264 (s)

[0.009814 0.009204 0.009264] 

ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.045498 (s)
sycl time 0.047151 

In [17]:
timeoneapi

array('d', [5.7e-05, 0.000111, 0.001005, 0.009204, 0.045498, 0.090369, 0.443393, 0.858119])

In [18]:
Stimeoneapi, Sstdoneapi = run_benchmark(buildoneapi, Ssizes, testname = "SInvariantMasses", nruns = 3, btype = 'sycl')

ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000226 (s)
sycl time 4.7e-05 (s)
sycl time 0.000128 (s)

[2.26e-04 4.70e-05 1.28e-04] 

ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.00028 (s)
sycl time 7.8e-05 (s)
sycl time 0.000165 (s)

[2.80e-04 7.80e-05 1.65e-04] 

ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000761 (s)
sycl time 0.000542 (s)
sycl time 0.000514 (s)

[0.000761 0.000542 0.000514] 

ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.005187 (s)
sycl time 0.004785 (s)
sycl time 0.004683 (s)

[0.005187 0.004785 0.004683] 

ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
S

In [19]:
timeopensycl, stdopensycl = run_benchmark(buildopensycl, sizes,  nruns = 3, btype = 'sycl')

ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
[hipSYCL Warning] OpenCL device Intel(R) FPGA Emulation Device does not support Intel USM extensions; falling back to fine-grained system SVM. USM pointer info queries have limited support.
[hipSYCL Warning] OpenCL device NVIDIA GeForce RTX 3060 does not have a valid USM provider. Memory allocations are not possible on that device.
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.001172 (s)
sycl time 8.7e-05 (s)
sycl time 6.9e-05 (s)

[1.172e-03 8.700e-05 6.900e-05] 

ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
[hipSYCL Warning] OpenCL device Intel(R) FPGA Emulation Device does not support Intel USM extensions; falling back to fine-grained system SVM. USM pointer info queries have limited support.
[hipSYCL Warning] OpenCL device NVIDIA GeForce RTX 3060 does not have a valid USM provider. Memory allocations are not possible on that device.
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl tim

In [20]:
timeopensycl

array('d', [6.9e-05, 0.00016, 0.001078, 0.010012, 0.05059, 0.10205, 0.482817, 0.947915])

In [21]:
Stimeopensycl, Sstdopensycl = run_benchmark(buildopensycl, Ssizes, testname = "SInvariantMasses", nruns = 3, btype = 'sycl')

ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
[hipSYCL Warning] OpenCL device Intel(R) FPGA Emulation Device does not support Intel USM extensions; falling back to fine-grained system SVM. USM pointer info queries have limited support.
[hipSYCL Warning] OpenCL device NVIDIA GeForce RTX 3060 does not have a valid USM provider. Memory allocations are not possible on that device.
sycl::queue check - selected device:
NVIDIA GeForce RTX 3060
sycl time 0.000829 (s)
sycl time 8.2e-05 (s)
sycl time 6.7e-05 (s)

[8.29e-04 8.20e-05 6.70e-05] 

ROOT_MEAS_TIMING defined 
ROOT_MATH_SYCL defined 
SINGLE_PRECISION defined 
[hipSYCL Warning] OpenCL device Intel(R) FPGA Emulation Device does not support Intel USM extensions; falling back to fine-grained system SVM. USM pointer info queries have limited support.
[hipSYCL Warning] OpenCL device NVIDIA GeForce RTX 3060 does not have a valid USM provider. Memory allocations are not possible on that device.
sycl::queue check - 

In [33]:
timecuda, stdcuda = run_benchmark(buildcuda, sizes, nruns = 3, btype = 'cuda')

ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.000285 (s)
cuda time 4.5e-05 (s)
cuda time 4.3e-05 (s)

[2.85e-04 4.50e-05 4.30e-05] 

ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.000462 (s)
cuda time 0.000198 (s)
cuda time 0.000183 (s)

[0.000462 0.000198 0.000183] 

ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.001821 (s)
cuda time 0.001416 (s)
cuda time 0.001323 (s)

[0.001821 0.001416 0.001323] 

ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.012467 (s)
cuda time 0.011873 (s)
cuda time 0.011905 (s)

[0.012467 0.011873 0.011905] 

ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.058219 (s)
cuda time 0.059817 (s)
cuda time 0.057277 (s)

[0.058219 0.059817 0.057277] 

ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
CUDA available devices: 1
cuda time 0.114114 (s)
cuda time 0.12278 

In [34]:
timecuda

array('d', [4.3e-05, 0.000183, 0.001323, 0.011873, 0.057277, 0.114114, 0.554021, 1.10137])

In [26]:
Stimecuda, Sstdcuda = run_benchmark(buildcuda, Ssizes, nruns = 3, testname = "SInvariantMasses", btype = 'cuda')

ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
SINGLE_PRECISION defined 
CUDA available devices: 1
cuda time 0.000227 (s)
cuda time 3e-05 (s)
cuda time 2.8e-05 (s)

[2.27e-04 3.00e-05 2.80e-05] 

ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
SINGLE_PRECISION defined 
CUDA available devices: 1
cuda time 0.000284 (s)
cuda time 8.5e-05 (s)
cuda time 8.6e-05 (s)

[2.84e-04 8.50e-05 8.60e-05] 

ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
SINGLE_PRECISION defined 
CUDA available devices: 1
cuda time 0.000868 (s)
cuda time 0.000693 (s)
cuda time 0.000667 (s)

[0.000868 0.000693 0.000667] 

ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
SINGLE_PRECISION defined 
CUDA available devices: 1
cuda time 0.005347 (s)
cuda time 0.004856 (s)
cuda time 0.004879 (s)

[0.005347 0.004856 0.004879] 

ROOT_MEAS_TIMING defined 
ROOT_MATH_CUDA defined 
SINGLE_PRECISION defined 
CUDA available devices: 1
cuda time 0.023098 (s)
cuda time 0.022807 (s)
cuda time 0.023466 (s)

[0.023098 0.022807 0.0

In [27]:
n = len(sizes) 
speedupopensycl = array('d', [timecpu[i]/timeopensycl[i] for i in range(n)])
speeduponeapi   = array('d', [timecpu[i]/timeoneapi[i] for i in range(n)])
speedupcuda     = array('d', [timecpu[i]/timecuda[i] for i in range(n)])

In [28]:
n = len(Ssizes)
Sspeedupopensycl = array('d', [Stimecpu[i]/Stimeopensycl[i] for i in range(n)])
Sspeeduponeapi   = array('d', [Stimecpu[i]/Stimeoneapi[i] for i in range(n)])
Sspeedupcuda     = array('d', [Stimecpu[i]/Stimecuda[i] for i in range(n)])

In [63]:
print(Sspeedupopensycl)
print(Sspeeduponeapi)
print(Sspeedupcuda)

array('d', [0.053023255813953486, 0.651492312330419, 3.3272453398606667, 5.228473062953996, 6.753687273007524, 7.4462361050998105, 7.955063352465417, 8.21675758488577, 8.312287411166908])
array('d', [0.08511697361871579, 1.3455790784557908, 4.598230549050221, 6.504706768332863, 7.675202539960588, 7.970513457931252, 8.38848960643532, 8.613883003820925, 8.80484603735487])
array('d', [0.000916118870441505, 0.012555121106663337, 0.10629684435942782, 0.7303151686818299, 2.9306643033614397, 4.624390707615049, 8.390163788574114, 9.309325663321593, 9.736042472106574])


In [31]:
val = list(sizes)
sizes_gb = [v*8*4/1e9 for v in val] #(nb of doubles)*(bytes for doubles)*(4=dim of LVector)/(bytes in GB)
sizes_gb = array('d', sizes_gb)
sizes_gb

array('d', [3.2e-05, 0.00032, 0.0032, 0.032, 0.16, 0.32, 1.6, 3.2])

In [37]:
val = list(Ssizes)
Ssizes_gb = [v*4*4/1e9 for v in val] #(nb of floats)*(bytes for floats)*(4=dim of LVector)/(bytes in GB)
Ssizes_gb = array('d', Ssizes_gb)
Ssizes_gb

array('d', [1.6e-05, 0.00016, 0.0016, 0.016, 0.08, 0.16, 0.8, 1.6, 3.2])

In [29]:
p = ROOT.TGraph(len(sizes), sizes, speedupopensycl)
g = ROOT.TGraph(len(sizes), sizes, speeduponeapi)
q = ROOT.TGraph(len(sizes), sizes, speedupcuda)


c4 = ROOT.TCanvas()
g.SetMarkerStyle(7)
g.SetLineColor(ROOT.kBlue)
p.SetMarkerStyle(5)
p.SetLineColor(ROOT.kRed)
q.SetMarkerStyle(3)
q.SetLineColor(ROOT.kGreen)


q.SetTitle("Speedup wrt RVec")
q.GetXaxis().SetTitle( 'Input size (# doubles)' )
q.GetYaxis().SetTitle( 'Speedup' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(p,"OpenSYCL","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")


c4.SetLogx();

q.Draw()
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("speedup.png")

Info in <TCanvas::Print>: png file speedup.png has been created


In [53]:
len(Ssizes), Ssizes, len(Sspeedupopensycl)

(9,
 array('d', [1000.0, 10000.0, 100000.0, 1000000.0, 5000000.0, 10000000.0, 50000000.0, 100000000.0, 200000000.0]),
 8)

In [30]:
p = ROOT.TGraph(len(Ssizes), Ssizes, Sspeedupopensycl)
g = ROOT.TGraph(len(Ssizes), Ssizes, Sspeeduponeapi)
q = ROOT.TGraph(len(Ssizes), Ssizes, Sspeedupcuda)


c4 = ROOT.TCanvas()
g.SetMarkerStyle(7)
g.SetLineColor(ROOT.kBlue)
p.SetMarkerStyle(5)
p.SetLineColor(ROOT.kRed)
q.SetMarkerStyle(3)
q.SetLineColor(ROOT.kGreen)


q.SetTitle("Speedup wrt RVec - Single Precision")
q.GetXaxis().SetTitle( 'Input size (# floats)' )
q.GetYaxis().SetTitle( 'Speedup' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(p,"OpenSYCL","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")


c4.SetLogx();

q.Draw()
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("Sspeedup.png")

Info in <TCanvas::Print>: png file Sspeedup.png has been created


In [31]:
p = ROOT.TGraphErrors(len(sizes), sizes, timeopensycl, stdopensycl)
g = ROOT.TGraphErrors(len(sizes), sizes, timeoneapi, stdoneapi)
q = ROOT.TGraphErrors(len(sizes), sizes, timecuda, stdcuda)
r = ROOT.TGraphErrors(len(sizes), sizes, timecpu, stdcpu)


c4 = ROOT.TCanvas()
g.SetLineColor(ROOT.kBlue)
p.SetLineColor(ROOT.kRed)
q.SetLineColor(ROOT.kGreen)


r.SetTitle("Execution times")
r.GetXaxis().SetTitle( 'Input size (# doubles)' )
r.GetYaxis().SetTitle( 'Time (s)' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(r,"RVec","L")
leg.AddEntry(p,"OpenSYCL","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")

c4.SetLogy();
c4.SetLogx();

r.Draw()
q.Draw('LP')
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("exectimes.png")

Info in <TCanvas::Print>: png file exectimes.png has been created


In [32]:
p = ROOT.TGraphErrors(len(sizes), sizes, Stimeopensycl, Sstdopensycl)
g = ROOT.TGraphErrors(len(sizes), sizes, Stimeoneapi, Sstdoneapi)
q = ROOT.TGraphErrors(len(sizes), sizes, Stimecuda, Sstdcuda)
r = ROOT.TGraphErrors(len(sizes), sizes, Stimecpu, Sstdcpu)


c4 = ROOT.TCanvas()
g.SetLineColor(ROOT.kBlue)
p.SetLineColor(ROOT.kRed)
q.SetLineColor(ROOT.kGreen)


r.SetTitle("Execution times - Single Precision")
r.GetXaxis().SetTitle( 'Input size (# floats)' )
r.GetYaxis().SetTitle( 'Time (s)' )

p.GetYaxis().SetRangeUser(0,6)

leg = ROOT.TLegend(0.7, 0.75, 0.30, 0.87)
leg.AddEntry(r,"RVec","L")
leg.AddEntry(p,"OpenSYCL","L")
leg.AddEntry(g,"OneAPI","L")
leg.AddEntry(q,"CUDA","L")

c4.SetLogy();
c4.SetLogx();

r.Draw()
q.Draw('LP')
g.Draw('LP')
p.Draw('LP')
leg.Draw()
c4.Draw()

c4.SaveAs("Sexectimes.png")

Info in <TCanvas::Print>: png file Sexectimes.png has been created
